<a href="https://colab.research.google.com/github/chayhuixiang/sf-pokemon-data-analytics/blob/main/%5BParticipants%5D_MLDA_Visualisation_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analytics (HX)

We will be using a open source dataset of Pokemon Go Spawns at the San Francisco Bay Area, from [Kaggle](https://www.kaggle.com/datasets/kveykva/sf-bay-area-pokemon-go-spawns).

Using this dataset, we will attempt to analyse for various data insights and obtain a comprehensive description of the data by investigating the distribution of pokemon by various factors (Name, Type, Time etc.)

## Extracting the dataset from Google Drive

Run these cells to download the dataset!

In [ ]:
# Download the csv file from google drive into the sample_data folder
! wget "https://drive.google.com/u/0/uc?id=1NGvsf9pwkXW_3G6xM7ar-OhnviYNjFKJ&export=download" -O "pokemon-spawns.csv" 
! mv pokemon-spawns.csv /content/sample_data/

## Data Pre-Processing
1. Read the csv file we have using the Pandas library

2. Display the dataframe obtained

3. Optimise memory usage

4. Filter unnecessary information, noisy data and perform miscellaneous data appending or slicing

In [ ]:
# Converting the csv file into a pandas dataframe
import pandas as pd
import numpy as np

raw_df = pd.read_csv('sample_data/pokemon-spawns.csv')

In [ ]:
# Sampling random indexes of the dataframe to see the kind of data we are dealing with
print(raw_df.shape)
raw_df.sample(5)

In [ ]:
# Retrieving all Pidgey spawns
raw_df[raw_df["name"] == "Pidgey"]

In [ ]:
# Checking for weird data with invalid encounter_ms
raw_df[raw_df["encounter_ms"] <= 0]

In [ ]:
# Column rename and checking for weird data with invalid disappear_ms
raw_df.rename(columns={"disppear_ms":"disappear_ms"}, inplace=True)
raw_df[raw_df["disappear_ms"] <= 0]

In [ ]:
# Retrieving all Nidoran ♀ spawns
raw_df[(raw_df['num'] == 29)]

In [ ]:
# Retrieving all Nidoran ♂ spawns
raw_df[(raw_df['num'] == 32)]

Interesting Points to note:

1. There exists datapoints with encounter_ms = -1, but there exists no datapoints with disappear_ms = -1.

2. The timestamps of encounter_ms are later than timestamps of disappear_ms.

3. 2 different name labels for Nidoran (eg. Nidoran♂ and Nidoran (m))

4. There is a lack of information regarding pokemon types.

Assumptions / Conclusions:

1. Datapoints with encounter_ms = -1 indicate pokemon that weren't encountered by Pokemon Trainers (ie. left to despawn)

2. Mislabelled data? disappear_ms should be spawn_ms instead.

3. Duplicate labels for Nidoran

In [ ]:
# Ralabelling encounter_ms = -1
raw_df['encounter_ms'].replace(-1, np.nan, inplace=True)
raw_df

In [ ]:
# Relabelling to spawn_ms
raw_df.rename(columns={"disappear_ms":"spawn_ms"}, inplace=True)
raw_df.head()

In [ ]:
# Fixing the Nidoran issue
raw_df['name'].replace('Nidoran (f)','Nidoran♀', inplace=True)
raw_df[(raw_df['num'] == 29)]

In [ ]:
# A more foolproof way of fixing the Nidoran issue + Add Pokemon Types
type_df = pd.read_csv("https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv")
type_df

In [ ]:
# Filtering the type df to get the information that we need
type_df = type_df.loc[type_df['Generation'] == 1, ['#','Name','Type 1','Type 2']].drop_duplicates(subset=['#'])
type_df.head()

In [ ]:
# Adding types to the dataframe, and rearranging the dataframe columns
raw_df = raw_df.merge(type_df,how='left',left_on='num',right_on='#').drop(columns=['#','name'])
cols = list(raw_df)
cols = cols[:3] + cols[-3:] + cols[3:7]
raw_df = raw_df[cols]
raw_df

In [ ]:
# Lowercase Column Names for linguistic Consistency
cols = list(raw_df)
cols = [name.lower() for name in cols]
raw_df.columns = cols
raw_df.head()

In [ ]:
# Information about the datatype used for every column
raw_df.info()

More information on S2 Cells:
https://pokemongohub.net/post/article/comprehensive-guide-s2-cells-pokemon-go/amp/

Pokemon Pokedex:
https://www.pokemon.com/us/pokedex/

In [ ]:
# Dropping information about S2 Cells
raw_df.drop(columns = ["s2_id","s2_token"], inplace=True)
raw_df.head()

In [ ]:
# Optimising data storage
raw_df['num'] = raw_df['num'].astype('uint16')
raw_df['name'] = raw_df['name'].astype('category')
raw_df['type 1'] = raw_df['type 1'].astype('category')
raw_df['type 2'] = raw_df['type 2'].astype('category')
raw_df['lat'] = raw_df['lat'].astype('float32')
raw_df['lng'] = raw_df['lng'].astype('float32')
raw_df['encounter_ms'] = pd.to_datetime(raw_df['encounter_ms'], unit='ms')
raw_df['spawn_ms'] = pd.to_datetime(raw_df['spawn_ms'], unit='ms')

In [ ]:
raw_df.info()

In [ ]:
raw_df.sample(5)

In [ ]:
raw_df.describe()

In [ ]:
df = raw_df.copy()

## Google Colab Data Tables

Colab includes an extension that renders pandas dataframes into interactive displays that can be filtered, sorted, and explored dynamically.

Here, we will be using this built in google colab feature to further filter and refine our analysis.

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
# Taking another look at our processed dataframe
df

In [ ]:
# Filtering 20000 random indexes from our dataframe and analysing using Colab Tables
df.sample(20000)

In [ ]:
# Challenge 1: Look for any anomalous data not found near the San Francisco region; determine the location of these anomalous points

In [ ]:
data_table.disable_dataframe_formatter()

## Seaborn & Matplotlib

Seaborn and Matplotlib are 2 common libraries used to visualise dataframes in a graphical or chart format. 

Both libraries are sufficient for most use cases. 

Here, we will be deriving various useful from our data using visualisation, such as the most common & least common pokemon spawns at the San Francisco Bay Area.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Taking another look at our processed dataframe
df

In [ ]:
# Count plot of all pokemon
df.num.value_counts().plot(kind = 'bar')

In [ ]:
# Count plot of the 10 most common pokemon
df.name.value_counts().sort_values()[-10:].plot(kind = 'bar')

In [ ]:
# Challenge 2: Count plot of the 10 least common pokemon

In [ ]:
# Count plot of different types
counts = df['type 1'].value_counts().add(df['type 2'].value_counts(), fill_value = 0)
counts.plot(kind='bar')

In [ ]:
# Normalized Probabilities Plot
(100*counts/counts.sum()).plot(kind='bar')

In [ ]:
# Spawn duration
spawn = df[df['encounter_ms'].notnull()]['encounter_ms'] - df[df['encounter_ms'].notnull()]['spawn_ms']
sns.histplot(spawn.astype('timedelta64[s]')/60)

In [ ]:
sns.histplot(data=df,x='encounter_ms')

In [ ]:
# Scatterplot of encounter times of different pokemon
sns.scatterplot(data=df,x='encounter_ms',y='num')

In [ ]:
# Geospatial analysis of plots
df.plot(kind='scatter', x='lng', y='lat')

In [ ]:
# Selecting only points at San Francisco
sanfrancisco_df = df[(df['lng'] <= -121.674) & (df['lat'] >= 37.196)]
sanfrancisco_df.plot(kind = 'scatter', x = 'lng', y='lat')

In [ ]:
sanfrancisco_df.plot(kind='hexbin', x='lng', y='lat', gridsize=20, colormap='binary')

In [ ]:
# Using Seaborn to generate a hexbin jointplot
sns.jointplot(x=sanfrancisco_df['lng'],y=sanfrancisco_df['lat'], kind='hex', joint_kws={'gridsize':20})

## Plot.ly
One caveat for both Seaborn & Matplotlib is the lack of interactivity (ability to zoom in to graphs, pan graph windows, and display information about datapoints when hovering over them). Plotly provides a viable, yet relatively high performance alternative. 

In [ ]:
import plotly.express as px

In [ ]:
# Taking another look at our processed dataframe
df

In [ ]:
# Plotting a count plot of pokemon types using plotly
fig = px.bar(x=counts.index, y=counts, labels={'y':'Counts','x':'Pokemon Types'}, color=counts.index, title='Histogram of Pokemon Types')
fig.show()

In [ ]:
# Plotly Histogram
fig = px.histogram(df,x='encounter_ms')
fig.show()

In [ ]:
# Interactive scatterplot with plotly
fig = px.scatter(df.sample(30000), x="encounter_ms", y="num")
fig.show()

In [ ]:
fig = px.scatter(sanfrancisco_df.sample(30000),x='lng',y='lat',hover_name='name',color='num',hover_data=['spawn_ms'])
fig.show()

## Kepler.gl

Plotting geospatial data using Matplotlib is viable using a traditional scatterplot or a hexbin plot, but there offers way more efficient and useful tools specifically engineered to handle such geospatial datasets. 

Kepler.gl is a powerful open source geospatial analysis tool for large-scale data sets, that offers both visual appeal and a broad range of functionality including interactivity, filters and different map types. 

For more information, visit their [GitHub](https://github.com/keplergl/kepler.gl)

Here, we will be importing our dataset into keplergl, and exploring the various functionality within this tool.

In [ ]:
! pip install keplergl

In [ ]:
from keplergl import KeplerGl
# to view Kepler GL output
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Drop unnecessary data to avoid bloating
spawn_map_df = df.drop(columns=['encounter_ms','type 1','type 2'])

In [ ]:
spawn_map_df.head()

In [ ]:
# Displaying a map of all spawns
spawn_map = KeplerGl(height=400)
spawn_map.add_data(data=spawn_map_df.sample(30000), name="Spawn Map Data")
spawn_map

In [ ]:
# Drop unnecessary data to avoid bloating
unencountered_map_df = df[df['encounter_ms'].isnull()].drop(columns={'encounter_ms'})
print(unencountered_map_df.shape)
unencountered_map_df

In [ ]:
# Challenge 3: Displaying a map of all unencountered pokemon

In [ ]:
# Using the kepler.gl webapp
unencountered_map_df.to_csv('unencountered_map.csv')
spawn_map_df.sample(30000).to_csv('spawn_map.csv')

## Insights gained:
1. Anomalous data found at **Tokyo** region
2. Top 10 most common pokemon: **Pidgey, Zubat, Rattata, Spearow, Weedle, Doduo, Ekans, Paras, Eevee, Magikarp**
3. Top 10 least common pokemon: **Omastar, Muk, Gyarados, Vaporeon, Cloyster, Machamp, Vileplume, Dewgong, Victreebel, Alakazam**
4. Most common spawn types: **Normal, Flying, Poison**
5. Spawn Duration: **15mins**
6. Dataset clustered on **26 July 2016**, encounters peak at **18:20**
7. **No underlying hotspots** for unencountered pokemon